In [4]:
# import statements
import numpy as np
import xarray as xr
import glob
import random
import PolarTestingTrainingSplit_CV
import tensorflow as tf
from tensorflow import keras
import keras_tuner

# Get names of models in which we are testing on
path_to_data = '/home/disk/pna2/aodhan/SurfaceTrendLearning/PoChedleyEtAl2022/TASmaps/*_TrendMaps.nc'
ModelNames = [i[70:-16] for i in glob.glob(path_to_data)]

# Do CV train-test-split 
TrainingPredictorData, TrainingTargetData, TestingPredictorData, TestingTargetData, TestingTotalTrend = PolarTestingTrainingSplit_CV.training_testing_split(path_to_data)


In [5]:
# Reshape target and predictor data 
model_idx = 7
TrainingTargetDataShape = np.shape(TrainingTargetData[model_idx])
TestinTargetDataShape = np.shape(TestingTargetData[model_idx])
TrainingTargetDataReshaped = np.reshape(TrainingTargetData[model_idx], (TrainingTargetDataShape[0], TrainingTargetDataShape[1]*TrainingTargetDataShape[2]))
TestingTargetDataReshaped = np.reshape(TestingTargetData[model_idx], (TestinTargetDataShape[0], TestinTargetDataShape[1]*TestinTargetDataShape[2]))
TrainingTargetDataReshaped = np.transpose([TrainingTargetDataReshaped[:,0], TrainingTargetDataReshaped[:,3]])
TestingTargetDataReshaped = np.transpose([TestingTargetDataReshaped[:,0], TestingTargetDataReshaped[:,3]])
TrainingPredictorDataReshaped = np.reshape(TrainingPredictorData[model_idx], (len(TrainingPredictorData[model_idx]),72,144))
TestingPredictorDataReshaped = np.reshape(TestingPredictorData[model_idx], (len(TestingPredictorData[model_idx]),72,144))

print(np.shape(TrainingPredictorDataReshaped), np.shape(TestingPredictorDataReshaped), np.shape(TrainingTargetDataReshaped), np.shape(TestingTargetDataReshaped))

(8268, 72, 144) (50, 72, 144) (8268, 2) (50, 2)


In [6]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=8, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [2,3,4,5]),
        activation='relu',
        input_shape=(72,144,1)
    ),
    keras.layers.MaxPooling2D(
        pool_size=hp.Choice('poolsize', values=[2,3,4,5])
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=4, max_value=32, step=4),
    ),
    keras.layers.Dense(2)
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
              loss='mean_absolute_error',
              metrics=['mean_absolute_error'])
  
  return model

In [7]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [8]:
tuner_search=RandomSearch(build_model,
                          objective='val_mean_absolute_error',
                          max_trials=20, overwrite=True, directory='/home/disk/p/aodhan/SurfaceTrendLearing/TropicalTASApplication/output',project_name="CNN_trial1")
tuner_search.search(TrainingPredictorDataReshaped, TrainingTargetDataReshaped, epochs=4, validation_split=0.1)

Trial 20 Complete [00h 02m 15s]
val_mean_absolute_error: 0.012651288881897926

Best val_mean_absolute_error So Far: 0.012497637420892715
Total elapsed time: 00h 38m 26s
INFO:tensorflow:Oracle triggered exit


In [9]:
model=tuner_search.get_best_models(num_models=1)[0]

In [11]:
tuner_search.results_summary()

Results summary
Results in /home/disk/p/aodhan/SurfaceTrendLearing/TropicalTASApplication/output/CNN_trial1
Showing 10 best trials
Trial summary
Hyperparameters:
conv_1_filter: 120
conv_1_kernel: 3
poolsize: 2
dense_1_units: 8
learning_rate: 1e-05
Score: 0.012497637420892715
Trial summary
Hyperparameters:
conv_1_filter: 72
conv_1_kernel: 2
poolsize: 2
dense_1_units: 4
learning_rate: 0.0001
Score: 0.012651288881897926
Trial summary
Hyperparameters:
conv_1_filter: 40
conv_1_kernel: 5
poolsize: 4
dense_1_units: 32
learning_rate: 0.0001
Score: 0.012865300290286541
Trial summary
Hyperparameters:
conv_1_filter: 72
conv_1_kernel: 4
poolsize: 5
dense_1_units: 8
learning_rate: 0.0001
Score: 0.012995569035410881
Trial summary
Hyperparameters:
conv_1_filter: 104
conv_1_kernel: 3
poolsize: 4
dense_1_units: 20
learning_rate: 0.0001
Score: 0.013042193837463856
Trial summary
Hyperparameters:
conv_1_filter: 88
conv_1_kernel: 4
poolsize: 3
dense_1_units: 8
learning_rate: 0.0001
Score: 0.013124180957674